In [1]:
import numpy as np
import pandas as pd
rng = np.random.default_rng(12345)

from lymph.models import Unilateral
from lymixture import LymphMixture
from lymixture.utils import binom_pmf, late_binomial, normalize

dataset = pd.read_csv('../../data/mixture_model_data_combined.csv', header = [0,1,2],)
dataset_staging = dataset.copy()
dataset_staging['tumor','1','t_stage'] = dataset_staging['tumor','1','t_stage'].replace([0,1,2], 'early')
dataset_staging['tumor','1','t_stage'] = dataset_staging['tumor','1','t_stage'].replace([3,4], 'late')
dataset_staging = dataset_staging[~(dataset_staging['tumor']['1']['subsite'].str.startswith(('C00.4')))]

In [2]:
from lymixture.utils import map_to_simplex, map_to_unit_cube
vec = np.array([[0,4,3]])
simplex = map_to_simplex(vec)
simplex

array([[0.5       , 0.01798621, 0.04742587],
       [0.5       , 0.98201379, 0.95257413]])

In [4]:
normalizer = 1/ simplex[0]
unit_cube = np.log(simplex[1:] * normalizer)
unit_cube

array([[0., 4., 3.]])

In [5]:
non_normalized = np.zeros((vec.shape[0]+1, vec.shape[1]))
non_normalized[1:, :] = vec
non_normalized

array([[0., 0., 0.],
       [0., 4., 3.]])

In [3]:
dataset_staging_main_groups = dataset_staging.copy()

# Reduce to only main subsite
dataset_staging_main_groups.loc[:, ('tumor', '1', 'subsite')] = (
    dataset_staging_main_groups.loc[:, ('tumor', '1', 'subsite')].str.replace(r'\..*', '', regex=True)
)

dataset_staging_main_groups['tumor']['1']['subsite'].value_counts()


subsite
C09    452
C12    227
C01    212
C10    169
C13    165
C02    158
C04     99
C05     61
C06     46
C03     45
Name: count, dtype: int64

In [4]:
graph = {
    ("tumor", "T"): ["II", "III"],
    ("lnl", "II"): ["III"],
    ("lnl", "III"): [],
}
num_components = 3

mixture = LymphMixture(
    model_cls=Unilateral,
    model_kwargs={"graph_dict": graph},
    num_components=num_components,
)
mixture.load_patient_data(
    dataset_staging_main_groups,
    split_by=("tumor", "1", "subsite"),
    mapping=lambda x: x,
)


In [5]:
mixture.subgroups.keys()

dict_keys(['C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C09', 'C10', 'C12', 'C13'])

Set the diagnostic modality

In [5]:
mixture.set_modality("diagnostic_consensus", 1., 1.)
mixture.get_all_modalities()

{'diagnostic_consensus': Clinical(spec=1.0, sens=1.0, is_trinary=False)}

Fix the distribution over diagnosis times for early T-stage (T1 & T2) to be a binomial distribution with a parameters $p=0.3$.

The late T-stage's diagnosis time distribution is a binomial one with a free model parameter than needs to be learned as well.

In [6]:
mixture.set_distribution("early", binom_pmf(np.arange(11), 10, 0.3))
mixture.set_distribution("late", late_binomial)
mixture.get_all_distributions()

{'early': Distribution([0.0282475249, 0.121060821, 0.23347444050000002, 0.266827932, 0.20012094900000002, 0.1029193452, 0.03675690900000001, 0.009001692, 0.0014467005000000002, 0.00013778100000000004, 5.904900000000001e-06]),
 'late': Distribution([0.0009765625, 0.009765625, 0.0439453125, 0.1171875, 0.205078125, 0.24609375, 0.205078125, 0.1171875, 0.0439453125, 0.009765625, 0.0009765625])}

Initialize random model parameters and latent variables/responsibilities.

In [7]:
from lymixture.em import expectation, maximization

params = {k: rng.uniform() for k in mixture.get_params()}
# params = {'0_TtoII_spread': 0.9999351914363541,
#  '0_TtoIII_spread': 0.03366844031848883,
#  '0_IItoIII_spread': 0.10981994569039934,
#  '0_C01_coef': 0.7165386319536359,
#  '0_C02_coef': 0.08084847743429682,
#  '0_C03_coef': 6.610696135189607e-05,
#  '0_C04_coef': 0.2235994962129972,
#  '0_C05_coef': 0.28177880547736045,
#  '0_C06_coef': 0.03263619582993652,
#  '0_C09_coef': 0.5867182765303357,
#  '0_C10_coef': 0.297036560366182,
#  '0_C12_coef': 6.610696135189607e-05,
#  '0_C13_coef': 0.05433176618922295,
#  '1_TtoII_spread': 0.3144310246089535,
#  '1_TtoIII_spread': 0.2489290062367473,
#  '1_IItoIII_spread': 6.610696135189607e-05,
#  '1_C01_coef': 0.15050493538887488,
#  '1_C02_coef': 0.2081276579287246,
#  '1_C03_coef': 0.0,
#  '1_C04_coef': 0.03871211836732616,
#  '1_C05_coef': 0.10606250261145811,
#  '1_C06_coef': 0.036081640923520224,
#  '1_C09_coef': 0.16629204773948103,
#  '1_C10_coef': 0.36283557463802174,
#  '1_C12_coef': 0.8611005036195872,
#  '1_C13_coef': 0.6956256038018159,
#  '2_TtoII_spread': 0.060352887501073316,
#  '2_TtoIII_spread': 0.009783370072349671,
#  '2_IItoIII_spread': 6.610696135189607e-05,
#  '2_C01_coef': 0.13295643265748924,
#  '2_C02_coef': 0.7110238646369786,
#  '2_C03_coef': 0.9999338930386481,
#  '2_C04_coef': 0.7376883854196766,
#  '2_C05_coef': 0.6121586919111814,
#  '2_C06_coef': 0.9312821632465432,
#  '2_C09_coef': 0.24698967573018327,
#  '2_C10_coef': 0.3401278649957963,
#  '2_C12_coef': 0.13883338941906087,
#  '2_C13_coef': 0.2500426300089611,
#  'late_p': 0.4549390063482304}
mixture.set_params(**params)
mixture.normalize_mixture_coefs()
latent = normalize(rng.uniform(size=mixture.get_resps().shape).T, axis=0).T

In [12]:
mixture._mixture_coefs

,C01,C02,C03,C04,C05,C06,C09,C10,C12,C13
0,0.314359,0.397392,0.337056,0.182529,0.644955,0.382452,0.191296,0.442896,0.349874,0.187215
1,0.205538,0.249108,0.312598,0.008068,0.160711,0.298911,0.535845,0.213237,0.584371,0.415452
2,0.480103,0.353500,0.350345,0.809403,0.194334,0.318638,0.272859,0.343867,0.065755,0.397333


In [13]:
from lymixture.em import expectation, maximization, _set_params
asdf = rng.uniform(size=32)
asdf

array([0.40029306, 0.16748354, 0.08201918, 0.08379994, 0.61145778,
       0.33145698, 0.57379222, 0.22299055, 0.05244128, 0.85062595,
       0.0236338 , 0.18243594, 0.39785484, 0.69342231, 0.34450649,
       0.02569248, 0.15783418, 0.81556022, 0.60662785, 0.27937044,
       0.84317692, 0.25327168, 0.79649697, 0.49906458, 0.49274613,
       0.39618075, 0.60510555, 0.67620104, 0.30067761, 0.92842046,
       0.76414179, 0.53994131])

In [14]:
mix_coeff = asdf[12:]
mix_coeff

array([0.24016936, 0.3416174 , 0.66565844, 0.69138827, 0.9158941 ,
       0.1462463 , 0.39150283, 0.46665116, 0.7408203 , 0.00151844,
       0.24417837, 0.75553651, 0.7994488 , 0.26958634, 0.401138  ,
       0.75809046, 0.74806057, 0.41665456, 0.41427486, 0.16258921])

In [8]:
def to_numpy(params: dict[str, float]) -> np.ndarray:
    return np.array([p for p in params.values()])

Iterate the computation of the expectation value of the latent variables (E-step) and the maximization of the (complete) data log-likelihood w.r.t. the model parameters (M-step).

In [9]:
def check_convergence(params_history, likelihood_history, steps_back_list):
    current_params = params_history[-1]
    current_likelihood = likelihood_history[-1]
    for steps_back in steps_back_list:
        previous_params = params_history[-steps_back - 1]
        if np.allclose(to_numpy(current_params), to_numpy(previous_params)):
            print('stopped due to parameter similarity')
            return True  # Return True if any of the steps is close
        elif np.isclose(current_likelihood, likelihood_history[-steps_back - 1]):
            print('stopped due to likelihood similarity')
            return True
    return False

In [11]:
is_converged = False
count = 0
params_history = []
likelihood_history = []

# Number of steps to look back for convergence
look_back_steps = 3
params_history.append(params.copy())
likelihood_history.append(mixture.likelihood(use_complete = False))

while not is_converged:
    print(count)
    old_params = params
    latent = expectation(mixture, params)
    params = maximization(mixture, latent)
    print('old_params', mixture.likelihood(use_complete=True, given_params = old_params))
    print('new params', mixture.likelihood(use_complete=True, given_params = params))
    # Append current params and likelihood to history
    params_history.append(params.copy())
    likelihood_history.append(mixture.likelihood())
    
    # Check if converged
    if count >= 3:  # Ensure enough history is available
        is_converged = check_convergence(params_history, likelihood_history,list(range(1,look_back_steps+1)))
    
    count += 1
params_history.append(params.copy())
likelihood_history.append(mixture.likelihood(use_complete = False))

0


In [137]:
likelihood_history

[-1840.8082463213277,
 -1838.5964374014243,
 -1845.1690193906538,
 -1837.2150494208781,
 -1843.9867124380098,
 -1836.8584368608394,
 -1844.1259934775985,
 -1836.6274275765693,
 -1843.832705901501,
 -1836.4653288886611,
 -1843.8156268762616]

In [142]:
likelihood_history

[-3808.707002948824,
 -1866.5608159211047,
 -1866.5608159211047,
 -1871.7652490270088,
 -1871.7652490270088,
 -1839.1926998293552,
 -1839.1926998293552,
 -1839.0957876469693,
 -1839.0957876469693,
 -1839.0130925961244,
 -1839.0130925961244,
 -1838.9350704230237,
 -1838.9350704230237,
 -1838.8565800896545,
 -1838.8565800896545,
 -1838.7758826565846,
 -1838.7758826565846,
 -1838.6925923554895,
 -1838.6925923554895,
 -1838.605736483376,
 -1838.605736483376,
 -1838.513912229501,
 -1838.513912229501,
 -1838.4159494001851,
 -1838.4159494001851,
 -1838.3096285511626,
 -1838.3096285511626,
 -1838.1897730724684,
 -1838.1897730724684,
 -1844.668860495503,
 -1844.668860495503,
 -1844.9523250778059,
 -1844.9523250778059,
 -1839.3928744968753,
 -1839.3928744968753,
 -1844.8698047342104,
 -1844.8698047342104,
 -1836.8879318859267,
 -1836.8879318859267,
 -1843.0883987113084,
 -1843.0883987113084,
 -1843.92288433564,
 -1843.92288433564,
 -1845.0938960729295,
 -1845.0938960729295,
 -1843.5668634630142,

In [135]:
params_history

[{'0_TtoII_spread': 0.8871230015148194,
  '0_TtoIII_spread': 6.610696135189607e-05,
  '0_IItoIII_spread': 0.16058087757136857,
  '0_C01_coef': 0.7353554599062526,
  '0_C02_coef': 0.23116548059175568,
  '0_C03_coef': 6.610696135189607e-05,
  '0_C04_coef': 3.850014114876546e-05,
  '0_C05_coef': 0.25779181410318286,
  '0_C06_coef': 0.009152031913958254,
  '0_C09_coef': 0.6046086377786678,
  '0_C10_coef': 0.33469106700801887,
  '0_C12_coef': 0.1153243962207408,
  '0_C13_coef': 0.1602474430856042,
  '1_TtoII_spread': 0.3496612945194357,
  '1_TtoIII_spread': 0.38513965585697396,
  '1_IItoIII_spread': 8.291381946198457e-06,
  '1_C01_coef': 0.11115709123859119,
  '1_C02_coef': 0.047024048609263125,
  '1_C03_coef': 0.0,
  '1_C04_coef': 0.09484258273851447,
  '1_C05_coef': 0.06322970019036828,
  '1_C06_coef': 0.02507105607128742,
  '1_C09_coef': 0.11792220213179472,
  '1_C10_coef': 0.25396319002243684,
  '1_C12_coef': 0.6415532747071483,
  '1_C13_coef': 0.4750361713097032,
  '2_TtoII_spread': 0.

In [136]:
params

{'0_TtoII_spread': 0.9999374531230925,
 '0_TtoIII_spread': 0.005524409093925671,
 '0_IItoIII_spread': 0.14535863390696943,
 '0_C01_coef': 0.7135949740356992,
 '0_C02_coef': 0.07519997731310805,
 '0_C03_coef': 6.610696135189607e-05,
 '0_C04_coef': 0.22059440074063127,
 '0_C05_coef': 0.2746828369407964,
 '0_C06_coef': 0.03126776011898311,
 '0_C09_coef': 0.5831542686134161,
 '0_C10_coef': 0.29459764419633216,
 '0_C12_coef': 6.610696135189606e-05,
 '0_C13_coef': 0.05688958141977362,
 '1_TtoII_spread': 0.31840664989309136,
 '1_TtoIII_spread': 0.25530558731417297,
 '1_IItoIII_spread': 6.610696135189607e-05,
 '1_C01_coef': 0.1522428712603201,
 '1_C02_coef': 0.20688465008228274,
 '1_C03_coef': 0.0,
 '1_C04_coef': 0.03953989490111362,
 '1_C05_coef': 0.1078522813535922,
 '1_C06_coef': 0.03610856474130164,
 '1_C09_coef': 0.1676090939477366,
 '1_C10_coef': 0.3602096442580276,
 '1_C12_coef': 0.8551453280846267,
 '1_C13_coef': 0.6861957398737709,
 '2_TtoII_spread': 0.06255274400275875,
 '2_TtoIII_sp

In [94]:
mixture.get_resps()

,0,1,2
0,0.867298,0.068498,0.064204
1,0.774648,0.221318,0.004034
2,0.867298,0.068498,0.064204
3,0.774648,0.221318,0.004034
4,0.894054,0.067773,0.038174
...,...,...,...
1629,0.053927,0.939107,0.006966
1630,0.004406,0.414287,0.581306
1631,0.130715,0.629267,0.240018
1632,0.000527,0.240581,0.758892


In [95]:
mixture.get_mixture_coefs()

,C01,C02,C03,C04,C05,C06,C09,C10,C12,C13
0,0.724130,0.105041,0.000066,0.000009,0.302517,0.075617,0.596715,0.311916,0.000066,0.066504
1,0.143275,0.185914,0.000000,0.153125,0.088429,0.001728,0.156986,0.350946,0.864009,0.687016
2,0.132595,0.709045,0.999934,0.846867,0.609054,0.922654,0.246299,0.337137,0.135925,0.246479


In [96]:
subgroup = 'C01'
print('true percentages for', subgroup)
print(mixture.subgroups[subgroup].patient_data['_model']['diagnostic_consensus'].value_counts()/mixture.subgroups[subgroup].patient_data['_model']['diagnostic_consensus'].value_counts().sum())
mixture.state_dist(subgroup=subgroup)
df = pd.DataFrame(np.array([mixture.state_dist(subgroup=subgroup)]), columns = [str(col) for col in mixture.components[0].graph.state_list])
print('predicted percentages for', subgroup)
print(df*100)

true percentages for C01
II     III  
True   False    0.518868
       True     0.325472
False  False    0.127358
       True     0.028302
Name: count, dtype: float64
predicted percentages for C01
      [0 0]     [0 1]      [1 0]     [1 1]
0  15.78346  2.673506  56.337814  25.20522


In [97]:
mixture.set_modality("diagnose", 1., 0.81 )

print(mixture.risk(subgroup='C01', involvement = {'II': True, 'III': None}, given_diagnosis={'diagnose':{'II': False, 'III': False}}))
mixture.del_modality("diagnose")

0.4161934109072689


In [98]:
mixture.risk(subgroup='C01', involvement = {'II': None, 'III': True}, given_diagnosis=None)

0.27878726348443644